In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from keras.models import Sequential
from keras.layers import Conv2D, Dropout, BatchNormalization, Flatten, Dense, MaxPooling2D
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam

from sklearn.metrics import mean_absolute_error, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('C:/Users/User/Downloads/age_gender.csv/age_gender.csv')
df['pixels'] = df['pixels'].map(lambda x: np.array(x.split(' '), dtype=np.float32).reshape(48, 48))

In [3]:
# Reshape images for CNN
X = np.array(df['pixels'].to_list())
X_reshaped = X.reshape(X.shape[0], 48, 48, 1)
# Normalise images
if np.max(X_reshaped) > 1: X_reshaped = X_reshaped / 255
# Set some useful variables
input_shape = X_reshaped.shape[1:] 

# Set y
y = df['age'].values
y_gender=df['gender'].values

In [4]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=123)

In [5]:
X_gtrain, X_gtest, y_gtrain, y_gtest = train_test_split(X_reshaped, y_gender, test_size=0.2, random_state=123)

# 1. Simple model

In [6]:
age_model_1 = Sequential()

age_model_1.add(Conv2D(64, kernel_size=(3,3), input_shape=input_shape, activation='relu'))
age_model_1.add(MaxPooling2D(pool_size=(2,2)))

age_model_1.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
age_model_1.add(MaxPooling2D(pool_size=(2,2)))

age_model_1.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
age_model_1.add(MaxPooling2D(pool_size=(2,2)))

age_model_1.add(Flatten())
age_model_1.add(Dense(128, activation='relu'))
age_model_1.add(Dropout(0.5))
age_model_1.add(Dense(1, activation='linear'))

age_model_1.compile(optimizer='adam', loss='mean_absolute_error')

age_model_1.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 256)         295168    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 4, 256)        0

In [7]:
# Callbacks for age model
callbacks = [EarlyStopping(patience=10, 
                               min_delta=0.001,
                               restore_best_weights=True),
             ReduceLROnPlateau(patience = 2,
                               verbose=1,
                               factor=0.5,
                               min_lr = 0.0001)]

In [ ]:
history = age_model_1.fit(X_train, y_train, 
                        epochs=35,
                        batch_size=64,
                        callbacks=callbacks,
                        validation_data=(X_test, y_test))

Epoch 1/35
297/297 [==============================] - 78s 260ms/step - loss: 15.9278 - val_loss: 14.3524 - lr: 0.0010
Epoch 2/35
297/297 [==============================] - 69s 231ms/step - loss: 12.9690 - val_loss: 11.6754 - lr: 0.0010
Epoch 3/35
297/297 [==============================] - 75s 251ms/step - loss: 11.3840 - val_loss: 10.3266 - lr: 0.0010
Epoch 4/35
297/297 [==============================] - 71s 238ms/step - loss: 10.9087 - val_loss: 9.9831 - lr: 0.0010
Epoch 5/35
297/297 [==============================] - 70s 236ms/step - loss: 10.0453 - val_loss: 10.0028 - lr: 0.0010
Epoch 6/35
297/297 [==============================] - 78s 261ms/step - loss: 9.6124 - val_loss: 8.8233 - lr: 0.0010
Epoch 7/35
297/297 [==============================] - 65s 219ms/step - loss: 9.0733 - val_loss: 10.3471 - lr: 0.0010
Epoch 8/35
297/297 [==============================] - 64s 217ms/step - loss: 8.7658 - val_loss: 7.8108 - lr: 0.0010
Epoch 9/35
297/297 [==============================] - 71s 240m

In [ ]:
pd.DataFrame(history.history).plot();

In [ ]:
from sklearn.metrics import r2_score

# Make predictions 
y_pred = age_model_1.predict(X_test)

# calculate R2 score
r2 = r2_score(y_test, y_pred)

print("R2 score: {:.3f}".format(r2))

In [ ]:
history=age_model_1.fit(X_gtrain,y_gtrain,epochs=35,
                        batch_size=64,
                        callbacks=callbacks,
                        validation_data=(X_gtest, y_gtest))

In [ ]:
pd.DataFrame(history.history).plot();

In [ ]:
from sklearn.metrics import r2_score

# Make predictions 
y_pred = age_model_1.predict(X_gtest)

# calculate R2 score
r2 = r2_score(y_gtest, y_pred)

print("R2 score: {:.3f}".format(r2))

# 2. Complex model

In [ ]:
age_model_2 = Sequential()

age_model_2.add(Conv2D(64, kernel_size=(3,3), input_shape=input_shape, activation='relu', padding='same'))
age_model_2.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
age_model_2.add(MaxPooling2D(pool_size=(2,2)))
age_model_2.add(BatchNormalization())

age_model_2.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
age_model_2.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
age_model_2.add(MaxPooling2D(pool_size=(2,2)))
age_model_2.add(Dropout(0.3))
age_model_2.add(BatchNormalization())

age_model_2.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
age_model_2.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
age_model_2.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
age_model_2.add(MaxPooling2D(pool_size=(2,2)))
age_model_2.add(Dropout(0.4))
age_model_2.add(BatchNormalization())

age_model_2.add(Conv2D(512, kernel_size=(3,3), activation='relu', padding='same'))
age_model_2.add(Conv2D(512, kernel_size=(3,3), activation='relu', padding='same'))
age_model_2.add(MaxPooling2D(pool_size=(2,2)))
age_model_2.add(Dropout(0.5))
age_model_2.add(BatchNormalization())

age_model_2.add(Flatten())
age_model_2.add(Dense(256, activation='relu'))
age_model_2.add(Dropout(0.3))
age_model_2.add(Dense(128, activation='relu'))
age_model_2.add(Dropout(0.2))
age_model_2.add(Dense(1))

age_model_2.compile(optimizer='adam', loss='mean_absolute_error')

age_model_2.summary()


In [ ]:
# Callbacks for age model
callbacks = [EarlyStopping(patience=10, 
                               min_delta=0.001,
                               restore_best_weights=True),
             ReduceLROnPlateau(patience = 2,
                               verbose=1,
                               factor=0.5,
                               min_lr = 0.0001)]

In [ ]:
history = age_model_2.fit(X_train, y_train, 
                        epochs=35,   
                        batch_size=64,
                        callbacks=callbacks,
                        validation_data=(X_test, y_test))

In [ ]:
pd.DataFrame(history.history).plot();

In [ ]:
from sklearn.metrics import r2_score

# Make predictions 
y_pred = age_model_2.predict(X_test)

# calculate R2 score
r2 = r2_score(y_test, y_pred)

print("R2 score: {:.3f}".format(r2))

In [ ]:
history=age_model_2.fit(X_gtrain,y_gtrain,epochs=35,
                        batch_size=64,
                        callbacks=callbacks,
                        validation_data=(X_gtest, y_gtest))

In [ ]:
pd.DataFrame(history.history).plot();

In [ ]:
from sklearn.metrics import r2_score

# Make predictions 
y_pred = age_model_2.predict(X_gtest)

# calculate R2 score
r2 = r2_score(y_gtest, y_pred)

print("R2 score: {:.3f}".format(r2))

# 3. Model with adjusted Flatten layer

In [ ]:
age_model_3 = Sequential()

age_model_3.add(Conv2D(64, kernel_size=(3,3), input_shape=input_shape, activation='relu'))
age_model_3.add(MaxPooling2D(pool_size=(2,2)))
age_model_3.add(BatchNormalization())

age_model_3.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
age_model_3.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
age_model_3.add(MaxPooling2D(pool_size=(2,2)))
age_model_3.add(Dropout(0.3))
age_model_3.add(BatchNormalization())

age_model_3.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
age_model_3.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
age_model_3.add(MaxPooling2D(pool_size=(2,2)))
age_model_3.add(Dropout(0.3))
age_model_3.add(BatchNormalization())

age_model_3.add(Conv2D(512, kernel_size=(3,3), activation='relu', padding='same'))
age_model_3.add(MaxPooling2D(pool_size=(2,2)))
age_model_3.add(Dropout(0.5))
age_model_3.add(BatchNormalization())

age_model_3.add(Flatten())
age_model_3.add(Dense(128, activation='relu'))
age_model_3.add(Dropout(0.4))
age_model_3.add(Dense(1))

age_model_3.compile(optimizer='adam', loss='mean_absolute_error')

age_model_3.summary()

In [ ]:
# Callbacks for age model
callbacks = [EarlyStopping(patience=10, 
                               min_delta=0.001,
                               restore_best_weights=True),
             ReduceLROnPlateau(patience = 2,
                               verbose=1,
                               factor=0.5,
                               min_lr = 0.0001)]

In [ ]:
history = age_model_3.fit(X_train, y_train, 
                        epochs=35,
                        batch_size=64,
                        callbacks=callbacks,
                        validation_data=(X_test, y_test))

In [ ]:
pd.DataFrame(history.history).plot();

In [ ]:
from sklearn.metrics import r2_score

# Make predictions 
y_pred = age_model_3.predict(X_test)

# calculate R2 score
r2 = r2_score(y_test, y_pred)

print("R2 score: {:.3f}".format(r2))

In [ ]:
history=age_model_3.fit(X_gtrain,y_gtrain,epochs=35,
                        batch_size=64,
                        callbacks=callbacks,
                        validation_data=(X_gtest, y_gtest))

In [ ]:
pd.DataFrame(history.history).plot();

In [ ]:
from sklearn.metrics import r2_score

# Make predictions 
y_pred = age_model_3.predict(X_gtest)

# calculate R2 score
r2 = r2_score(y_gtest, y_pred)

print("R2 score: {:.3f}".format(r2))

# 4.Model balanced

In [ ]:
age_model_4 = Sequential()

age_model_4.add(Conv2D(128, kernel_size=(2,2), input_shape=input_shape, activation='relu'))
age_model_4.add(MaxPooling2D(pool_size=(2,2)))
age_model_4.add(BatchNormalization())

age_model_4.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
age_model_4.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
age_model_4.add(MaxPooling2D(pool_size=(2,2)))
age_model_4.add(Dropout(0.3))
age_model_4.add(BatchNormalization())

age_model_4.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
age_model_4.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
age_model_4.add(MaxPooling2D(pool_size=(2,2)))
age_model_4.add(Dropout(0.3))
age_model_4.add(BatchNormalization())

age_model_4.add(Conv2D(512, kernel_size=(3,3), activation='relu', padding='same'))
age_model_4.add(MaxPooling2D(pool_size=(2,2)))
age_model_4.add(Dropout(0.5))
age_model_4.add(BatchNormalization())

age_model_4.add(Flatten())
age_model_4.add(Dense(64, activation='relu'))
age_model_4.add(Dropout(0.4))
age_model_4.add(Dense(32, activation='relu'))
age_model_4.add(Dropout(0.2))
age_model_4.add(Dense(1))

age_model_4.compile(optimizer='adam', loss='mean_absolute_error')

age_model_4.summary()


In [ ]:
# Callbacks for age model
callbacks = [EarlyStopping(patience=10, 
                               min_delta=0.001,
                               restore_best_weights=True),
             ReduceLROnPlateau(patience = 2,
                               verbose=1,
                               factor=0.5,
                               min_lr = 0.0001)]

In [ ]:
history = age_model_4.fit(X_train, y_train, 
                        epochs=35,
                        batch_size=64,
                        callbacks=callbacks,
                        validation_data=(X_test, y_test))

In [ ]:
pd.DataFrame(history.history).plot();

In [ ]:
from sklearn.metrics import r2_score

# Make predictions 
y_pred = age_model_4.predict(X_test)

# calculate R2 score
r2 = r2_score(y_test, y_pred)

print("R2 score: {:.3f}".format(r2))

In [ ]:
history=age_model_4.fit(X_gtrain,y_gtrain,epochs=35,
                        batch_size=64,
                        callbacks=callbacks,
                        validation_data=(X_gtest, y_gtest))

In [ ]:
pd.DataFrame(history.history).plot();

In [ ]:
from sklearn.metrics import r2_score

# Make predictions 
y_pred = age_model_4.predict(X_gtest)

# calculate R2 score
r2 = r2_score(y_gtest, y_pred)

print("R2 score: {:.3f}".format(r2))

In [ ]:
age_model_3.save('/kaggle/working/age_model_3.h5')
age_model_4.save('/kaggle/working/age_model_4.h5')
